# Treinamento WESAD no Google Colab (com GPU + Checkpoint + Git Sync)

Este notebook clona o repositório, configura o ambiente, treina o modelo (com checkpoint no Drive) e envia os resultados de volta para o GitHub.

In [ ]:
# 1. Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
DRIVE_CHECKPOINT_DIR = '/content/drive/MyDrive/wearable_checkpoints'
os.makedirs(DRIVE_CHECKPOINT_DIR, exist_ok=True)
print(f"Checkpoints serão salvos em: {DRIVE_CHECKPOINT_DIR}")

In [ ]:
# 2. Clonar Repositório
!git clone https://github.com/naubergois/wearable_research.git
%cd wearable_research
!ls

In [ ]:
# 3. Instalar Dependências
!pip install -q kaggle seaborn scikit-learn pandas numpy matplotlib tqdm torch

In [ ]:
# 4. Configurar API do Kaggle
from google.colab import files

if not os.path.exists("kaggle.json"):
    print("Por favor, faça o upload do arquivo kaggle.json")
    files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
print("Kaggle API configurada.")

In [ ]:
# 5. Configurar Checkpoints Persistentes
import glob

ckpts = ["training_checkpoint_mlp_features.pkl", "training_checkpoint_lstm_pytorch.pkl"]

for ckpt in ckpts:
    drive_path = os.path.join(DRIVE_CHECKPOINT_DIR, ckpt)
    local_path = os.path.join(os.getcwd(), ckpt)
    
    # Se existe no Drive, linka para o local (Resume)
    if os.path.exists(drive_path) and not os.path.exists(local_path):
        !ln -s "{drive_path}" "{local_path}"
        print(f"Linkado: {local_path} -> {drive_path}")

In [ ]:
# 6. Executar Pipeline Completo
# Treina -> Gera Figuras -> Matrizes de Confusao

!python train_models.py

# Backup de seguranca para o Drive
!cp training_checkpoint_*.pkl "{DRIVE_CHECKPOINT_DIR}/" 2>/dev/null || true
!cp training_results.json "{DRIVE_CHECKPOINT_DIR}/"

!python generate_figure_9.py
!python generate_figure_10.py
!python generate_figure_11.py
!python plot_confusion_matrices.py

In [ ]:
# 7. Atualizar Resultados no GitHub
from getpass import getpass

print("Para salvar os resultados no GitHub, insira seu Personal Access Token (PAT):")
print("(Crie um em https://github.com/settings/tokens se não tiver)")
token = getpass('GitHub PAT: ')

if token:
    # Configurar Git
    !git config --global user.email "colab@example.com"
    !git config --global user.name "Colab Runner"
    
    # Adicionar remote com token
    repo_url = "https://" + token + "@github.com/naubergois/wearable_research.git"
    !git remote set-url origin {repo_url}
    
    # Adicionar arquivos gerados
    !git add *.json *.png *_model.pkl *_model.pth
    
    # Commit e Push
    !git commit -m "chore: update results from Colab execution"
    !git push origin main
    print("Sucesso! Resultados enviados para o GitHub.")
else:
    print("Token não fornecido. Pulei o push.")